### 0. 事前準備

以下のデータを取得して、このノートブックと同じディレクトリにある`data`ディレクトリに配置します。

__COVID-19 World Vaccination Progress__  
URL:  
・https://www.kaggle.com/gpreda/covid-world-vaccination-progress    
DATA:  
・country_vaccinations.csv  
・country_vaccinations_by_manufacturer.csv

---

__Annotated geo-json geometry files for the world__  
URL:  
・https://github.com/johan/world.geo.json  
DATA:  
・countries.geo.json

In [ ]:
%matplotlib inline

import folium
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

### 1. データの概要を把握
まずデータの概要を掴んでみましょう。

国別ワクチン接種状況データをロード。

In [ ]:
country_vaccinations_df = pd.read_csv('data/country_vaccinations.csv')

len(country_vaccinations_df)

データの中身を確認。  
不明な属性があれば、データ取得元で確認します。

In [ ]:
country_vaccinations_df.head()

１行目のみを表示してみます。  
以下はどんな属性でしょうか？  
- `people_fully_vaccinated`
- `daily_vaccinations_raw`
- `daily_vaccinations`
- `people_vaccinated_per_hundred`

In [ ]:
country_vaccinations_df.iloc[0]

`source_name`、`source_website`列は今回必要ないので削除します。

In [ ]:
country_vaccinations_df.drop(columns={'source_name', 'source_website'}, inplace=True)

数値属性の集計情報を確認。  
どんな特徴が読み取れますか？

In [ ]:
country_vaccinations_df.describe()

ヒストグラムでデータを表示。  
どんな特徴が読み取れますか？  
※「1e9」は10の9乗の意味。

In [ ]:
country_vaccinations_df.hist(bins=20, figsize=(20, 12))

`daily_vaccinations_raw`と`daily_vaccinations`のグラフがかなり異なります。  
かなりの欠損値がありそうです。

`pandas.DataFrame.info()`で列ごとの欠損値を確認してみます。

In [ ]:
country_vaccinations_df.info()

### 2. 地図を使ってデータを把握

ここでは、各国をワクチン２回接種の割合で色分けしてみます。

各国ごとの最新のワクチン２回接種割合のデータフレームを作成。

In [ ]:
country_fully_vaccinated_df = country_vaccinations_df.groupby(['country', 'iso_code'])\
                                .max()['people_fully_vaccinated_per_hundred'].reset_index()

country_fully_vaccinated_df.head(20)

棒グラフで分布を確認。  
`seaborn.barplot()`でグラフを描画します。

In [ ]:
country_fully_vaccinated_from_top_df = \
    country_fully_vaccinated_df.sort_values('people_fully_vaccinated_per_hundred', ascending=False)
ax = sns.barplot(
    x=country_fully_vaccinated_from_top_df['country'],
    y=country_fully_vaccinated_from_top_df['people_fully_vaccinated_per_hundred'])

# x軸目盛りを表示しない。
ax.set(xticklabels=[])
ax.tick_params(bottom=False)

データを地図上に表示。  
`folium.Choropleth()`でコロプレスマップを描画。

国ごとの区画情報は`countries.geo.json`を利用します。  
参考： [foliumを使って東京23区ラーメンスコアを可視化する](https://qiita.com/xkent/items/d068fc504c8cef217ca8)  

どんなことが読み取れますか？

In [ ]:
map = folium.Map(location=[0,0], zoom_start=2.4)

folium.Choropleth(
    # geo_data=json.load(open('data/countries.geo.json', 'r')),
    geo_data='data/countries.geo.json',
    data=country_fully_vaccinated_df,
    fill_opacity=1,
    line_color='black',
    nan_fill_color='#888888',
    columns=['iso_code', 'people_fully_vaccinated_per_hundred'],
    key_on= "feature.id",
    fill_color='RdBu').add_to(map)

map

### 3. 少しだけ掘り下げた分析をしてみる
各製造メーカー製ワクチンの接種状況がどのように推移しているかを分析してみます。

製造メーカー別のワクチン接種データをロード。

In [ ]:
country_vaccinations_by_manufacturer_df = pd.read_csv('data/country_vaccinations_by_manufacturer.csv')

len(country_vaccinations_by_manufacturer_df)

データの中身を確認。

In [ ]:
country_vaccinations_by_manufacturer_df.head(20)

欠損値があるか確認。

In [ ]:
country_vaccinations_by_manufacturer_df.isna().sum()

グラフのX軸目盛用に`month`列を追加。

In [ ]:
country_vaccinations_by_manufacturer_df['month'] = pd.to_datetime(
    country_vaccinations_by_manufacturer_df.date, format='%Y-%m-%d').dt.strftime('%Y-%m')

ワクチン、月別に接種回数を合計したデータフレームを生成。

In [ ]:
vaccinations_by_manufacturer_df = country_vaccinations_by_manufacturer_df \
                                    .groupby(['vaccine', 'month']) \
                                    .agg({'total_vaccinations': 'sum'}) \
                                    .reset_index()

vaccinations_by_manufacturer_df.head()                                   

折れ線グラフを表示。  
どんなことが読み取れますか？

In [ ]:
ax = sns.lineplot(
    data=vaccinations_by_manufacturer_df,
    x='month',
    y='total_vaccinations',
    hue='vaccine',
    palette='Set1')
plt.xticks(rotation=90)
ax.legend(loc="lower left", bbox_to_anchor=(1, 0))